In [ ]:
# Etape 0.
# Importer et installer les dépendances
!pip install h3
!pip install geopandas

import pandas as pd
from h3 import h3
import geopandas
import json
import os
import io

In [9]:
# ETAPE 1a.
# Uploader manuellement le RPLS géolocalisé 2019 dans l'espace de travail Google Colab (colab.research.google.com)
# Lien pour télécharger le RPLS : data.gouv.fr/fr/datasets/repertoire-des-logements-locatifs-des-bailleurs-sociaux/

# ETAPE 1b.
# Concaténer l'ensemble des CSV (un par région) dans un unique dataframe
# et ne conserver que les colonnes utiles à la cartographie :
# la surface, la latitude et la longitude

files = os.listdir()

list = []

for file in files:
  if file.endswith('.csv'):
    list.append(pd.read_csv("/content/" + file,
                               sep=';',
                               low_memory=False,
                               usecols=['SURFHAB', 'latitude', 'longitude']))

df = pd.concat(list)

# Supprimer les données initiales pour préserver la RAM de l'espace de travail Google Colab
del list

In [ ]:
# ETAPE 2.
# Nettoyer les données

# Imprimer le nombre de lignes dans le fichier, correspond au
# nombre de logements sociaux en France d'après le RPLS utilisé
print("Nombre de LLS (avant nettoyage) :", len(df.index))

# Renommer les colonnes
df.rename(columns={
    'SURFHAB': 'SURFACE',
    'latitude': 'LATITUDE',
    'longitude': 'LONGITUDE'
    },
    inplace=True)

# Transformer les longitudes et latitudes en nombre
df['LONGITUDE'] = df['LONGITUDE'].replace(regex=r',', value='.')
df["LONGITUDE"] = pd.to_numeric(df["LONGITUDE"])
df['LATITUDE'] = df['LATITUDE'].replace(regex=r',', value='.')
df["LATITUDE"] = pd.to_numeric(df["LATITUDE"])

# Supprimer les logements où des surfaces, latitudes ou longitudes sont manquantes
df = df.dropna().reset_index()

# Créer une colonne contenant systématiquement 1
# Cette colonne permettra de sommer le nombre de logements plus bas
df["LOGEMENT"] = 1

# Créer une colonne contenant un nombre aléatoire afin de pouvoir
# disperser les logements sur un axe horizontal dans les graphiques
import random 
df["RANDOM"] = df.apply(lambda row: random.randint(0, 1000), axis = 1)

print("Nombre de LLS (après nettoyage) :", len(df.index))

In [ ]:
# ETAPE 3.
# Ajuster la surface des logements dont la valeur interroge

# Visualiser la dispersion des logements en fonction de leur surface
df.plot.scatter(x="RANDOM", y="SURFACE", c="#00000010")

# Pour les logements dont la surface est supérieure à 150 m²
# modifier leur surface en leur appliquant la surface médiane
median = df['SURFACE'].median()
df.loc[(df.SURFACE > 150), "SURFACE"]= median

# Visualiser la nouvelle dispersion des logements en fonction de leur surface
df.plot.scatter(x="RANDOM", y="SURFACE", c="#00000010")

In [ ]:
# ETAPE 4.
# Créer une nouvelle colonne contenant l'identifiant de l'Hexbin H3 de chaque logement (= chaque ligne)
# H3 est un sytème d'indexation géospatial développé par Uber : h3geo.org/docs/
# La résolution H3 utilisée est 11 -> h3geo.org/docs/core-library/restable

df["HEX_ID"] = df.apply(lambda row: h3.geo_to_h3(row["LATITUDE"], row["LONGITUDE"], 11), axis = 1)

# ETAPE 5.
# Grouper les données par HEX_ID et sommer les surfaces et le nombre de logements
df = df.groupby('HEX_ID', as_index=False).agg({
    "SURFACE": "sum",
    "LOGEMENT": "sum"})

print("Nombre d'Hexbins :", len(df.index))

In [ ]:
# ETAPE 6.
# Retirer les Hexbins dont les valeurs interrogent

# Visualiser la dispersion des Hexbins
df.plot.scatter(x="SURFACE", y="LOGEMENT", c="#00000010")

print("Nombre de LLS total (avant retrait):", df['LOGEMENT'].sum())

# Supprimer les Hexbins dont la surface cumulée est supérieure à 30 000 m²
df = df.drop(df[df.SURFACE > 30000].index)
df.reset_index()

print("Nombre de LLS total (après retrait):", df['LOGEMENT'].sum())

# Visualiser la nouvelle dispersion des Hexbins
df.plot.scatter(x="SURFACE", y="LOGEMENT", c="#00000010")

In [14]:
# ETAPE 7.
# Ajouter une colonne qui contient :
# - la géométrie de l'Hexbin au format Geojson
# - la surface cumulée des logements sociaux contenus dans l'Hexbin
# - le nombre de logements sociaux contenus dans l'Hexbin
df["geometry"] = df.apply(lambda row: {
    "type": "Feature",
    "geometry":{
        "type": "Polygon",
        "coordinates": [h3.h3_to_geo_boundary(row["HEX_ID"],geo_json=True)]
        },
        "properties": {"surface": row["SURFACE"], "count": row["LOGEMENT"]}
        }, 
        axis = 1)

In [15]:
# ETAPE 8.
# Finaliser la modélisation du fichier geojson de sortie

features = []
for i, row in df.iterrows():
  features.append(row['geometry'])
        
data = {
    "type": "FeatureCollection",
    "features": features 
    }

# Transformer le fichier en JSON valide       
data  = json.dumps(data)

In [ ]:
# ETAPE 9.
# Sauvegarder, puis télécharger les données finales

from google.colab import files

with open('data.geojson', 'w') as f:
  f.write(data)

files.download('data.geojson')